In [1]:
from importlib import reload
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./dracula.mplstyle')
import logging
LOG = logging.getLogger(__name__)
LOG.setLevel(logging.INFO)
LOG.info('Hello from notebook')

from pydataset import data

import frankenfit as ff
reload(ff.core)
reload(ff.transforms)
reload(ff.graph)
reload(ff)

logging.getLogger('frankenfit').setLevel(logging.INFO)

In [2]:
df = data('diamonds').reset_index().drop(['index'], axis=1)
ds = ff.PandasDataset(df)
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [3]:
df.groupby("cut")[["table", "depth"]].mean()

,table,depth
cut,,
Fair,59.053789,64.041677
Good,58.694639,62.365879
Ideal,55.951668,61.709401
Premium,58.746095,61.264673
Very Good,57.956150,61.818275


In [3]:
df.groupby("cut").apply(len)

cut
Fair          1610
Good          4906
Ideal        21551
Premium      13791
Very Good    12082
dtype: int64

In [4]:
transform = ff.DeMean(["table", "depth"])

In [5]:
ff.GroupBy("cut", transform)

GroupBy(tag='GroupBy#1', cols=['cut'], transform=DeMean(tag='DeMean#1', cols=['table', 'depth'], w_col=None), fitting_schedule=<function fit_group_on_self at 0x7f9129b39700>)

In [6]:
ff.GroupBy?

Init signature:
ff.GroupBy(
    cols: 'str | HP | Iterable[str | HP]',
    transform: 'ffc.HP | ffc.Transform',
    fitting_schedule: 'Callable' = <function fit_group_on_self at 0x7f9129b39700>,
    *,
    tag: 'str' = NOTHING,
) -> None
Docstring:     
Group the fitting and application of a :class:`Transform` by the distinct values of
some column or combination of columns.
Init docstring: Method generated by attrs for class GroupBy.
File:           ~/projects/frankenfit/src/frankenfit/graph.py
Type:           ABCMeta
Subclasses:     


In [229]:
fit_gb = ff.GroupBy("cut", transform).fit(df)

In [230]:
s = fit_gb.state()
s

,cut,__state__
0,Ideal,"FitDeMean(cols=['table', 'depth'], w_col=None, <21551 rows of fitting data>)"
1,Premium,"FitDeMean(cols=['table', 'depth'], w_col=None, <13791 rows of fitting data>)"
2,Good,"FitDeMean(cols=['table', 'depth'], w_col=None, <4906 rows of fitting data>)"
3,Very Good,"FitDeMean(cols=['table', 'depth'], w_col=None, <12082 rows of fitting data>)"
4,Fair,"FitDeMean(cols=['table', 'depth'], w_col=None, <1610 rows of fitting data>)"


In [231]:
len(df)

53940

In [232]:
result_df = fit_gb.apply(df)
result_df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,-0.209401,-0.951668,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,-1.464673,2.253905,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,-5.465879,6.305361,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,1.135327,-0.746095,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,0.934121,-0.694639,335,4.34,4.35,2.75


In [233]:
result_df[["table", "depth"]].mean()

table    2.375326e-15
depth   -4.944555e-15
dtype: float64

In [234]:
result_df.groupby("cut")[["table", "depth"]].mean()

,table,depth
cut,,
Fair,-5.825568e-16,-9.488614e-15
Good,-1.283206e-15,-1.257136e-15
Ideal,7.751315e-15,-2.420019e-15
Premium,-2.649787e-15,-1.010763e-14
Very Good,3.934391e-16,-4.456036e-15


In [19]:
pip = (
    ff.Pipeline()
    .stateless_lambda(len)
)
pip

Pipeline(tag='Pipeline#11', dataset_name='__pass__', transforms=[StatelessLambda(tag='StatelessLambda#4', apply_fun=<built-in function len>)])

In [20]:
pip.fit(df).apply(df)

53940

In [25]:
ff.GroupBy("cut", pip).fit(df).apply(df)

,cut,None
0,Ideal,21551
1,Premium,13791
2,Good,4906
3,Very Good,12082
4,Fair,1610


In [22]:
df.groupby("cut", as_index=False, sort=False).apply(len)

,cut,None
0,Ideal,21551
1,Premium,13791
2,Good,4906
3,Very Good,12082
4,Fair,1610


In [13]:
pip = (
    ff.Pipeline()
    .group_by("cut")
        .stateless_lambda(len)
)
pip

Pipeline(tag='Pipeline#7', dataset_name='__pass__', transforms=[GroupBy(tag='GroupBy#4', cols=['cut'], transform=StatelessLambda(tag='StatelessLambda#3', apply_fun=<built-in function len>), fitting_schedule=<function fit_group_on_self at 0x7f9129b39700>)])

In [14]:
pip.fit(df).apply(df)

,cut,None
0,Ideal,21551
1,Premium,13791
2,Good,4906
3,Very Good,12082
4,Fair,1610


In [15]:
df.groupby("cut")

In [17]:
gb = ff.Pipeline().group_by("cut")

In [18]:
gb

PipelineGrouper('cut', Pipeline(tag='Pipeline#9', dataset_name='__pass__', transforms=[]))

In [26]:
np.mean(df)

/home/max/projects/frankenfit/.venv-dev/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/max/projects/frankenfit/.venv-dev/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


carat       0.797940
depth      61.749405
table      57.457184
price    3932.799722
x           5.731157
y           5.734526
z           3.538734
dtype: float64

In [35]:
df.groupby("cut")[["price"]].describe()

price                                                             \
             count         mean          std    min      25%     50%      75%   
cut                                                                             
Fair        1610.0  4358.757764  3560.386612  337.0  2050.25  3282.0  5205.50   
Good        4906.0  3928.864452  3681.589584  327.0  1145.00  3050.5  5028.00   
Ideal      21551.0  3457.541970  3808.401172  326.0   878.00  1810.0  4678.50   
Premium    13791.0  4584.257704  4349.204961  326.0  1046.00  3185.0  6296.00   
Very Good  12082.0  3981.759891  3935.862161  336.0   912.00  2648.0  5372.75   

                    
               max  
cut                 
Fair       18574.0  
Good       18788.0  
Ideal      18806.0  
Premium    18823.0  
Very Good  18818.0

In [38]:
x = (
    ff.Pipeline()
    .group_by("cut")
    .de_mean(["price"])
    [["cut", "price"]]
).fit_and_apply(df)
x.groupby("cut")[["price"]].describe()

price                                                       \
             count          mean          std          min          25%   
cut                                                                       
Fair        1610.0  2.169230e-13  3560.386612 -4021.757764 -2308.507764   
Good        4906.0 -2.847501e-13  3681.589584 -3601.864452 -2783.864452   
Ideal      21551.0 -1.728593e-13  3808.401172 -3131.541970 -2579.541970   
Premium    13791.0  5.402495e-13  4349.204961 -4258.257704 -3538.257704   
Very Good  12082.0  0.000000e+00  3935.862161 -3645.759891 -3069.759891   

                                                   
                   50%          75%           max  
cut                                                
Fair      -1076.757764   846.742236  14215.242236  
Good       -878.364452  1099.135548  14859.135548  
Ideal     -1647.541970  1220.958030  15348.458030  
Premium   -1399.257704  1711.742296  14238.742296  
Very Good -1333.759891  1390.990109  14836.240109

In [7]:
x = (
    ff.Pipeline()
    .group_by("cut", fitting_schedule=ff.fit_group_on_all_other_groups)
        .de_mean(["price"])
    [["cut", "price"]]
).fit_and_apply(df)
x.groupby("cut")[["price"]].describe()

price                                                     \
             count        mean          std          min          25%   
cut                                                                     
Fair        1610.0  439.063191  3560.386612 -3582.694573 -1869.444573   
Good        4906.0   -4.329006  3681.589584 -3606.193458 -2788.193458   
Ideal      21551.0 -791.484860  3808.401172 -3923.026830 -3371.026830   
Premium    13791.0  875.230854  4349.204961 -3383.026850 -2663.026850   
Very Good  12082.0   63.092157  3935.862161 -3582.667734 -3006.667734   

                                                   
                   50%          75%           max  
cut                                                
Fair       -637.694573  1285.805427  14654.305427  
Good       -882.693458  1094.806542  14854.806542  
Ideal     -2439.026830   429.473170  14556.973170  
Premium    -524.026850  2586.973150  15113.973150  
Very Good -1270.667734  1454.082266  14899.332266

In [6]:
ff.Pipeline.group_by?

Signature: ff.Pipeline.group_by(self, cols, fitting_schedule=None) -> 'PipelineGrouper'
Docstring: <no docstring>
File:      ~/projects/frankenfit/src/frankenfit/graph.py
Type:      function


In [10]:
pip = (
    ff.Pipeline()
    .group_by("cut")
    .de_mean(["price"])
    [["cut", "price"]]
)
pip_fit = pip.fit(df)

In [11]:
pip

Pipeline(tag='Pipeline#9', dataset_name='__pass__', transforms=[GroupBy(tag='GroupBy#3', cols=['cut'], transform=DeMean(tag='DeMean#3', cols=['price'], w_col=None), fitting_schedule=<function fit_group_on_self at 0x7f175fb3f430>), Select(tag='Select#3', cols=['cut', 'price'])])

In [12]:
pip_fit

FitPipeline(tag='Pipeline#9', dataset_name='__pass__', transforms=[GroupBy(tag='GroupBy#3', cols=['cut'], transform=DeMean(tag='DeMean#3', cols=['price'], w_col=None), fitting_schedule=<function fit_group_on_self at 0x7f175fb3f430>), Select(tag='Select#3', cols=['cut', 'price'])], <53940 rows of fitting data>)

In [19]:
pip_fit.find_by_tag("GroupBy#3").state().loc[0,'__state__'].state()

price    3457.54197
dtype: float64

In [34]:
s = pip_fit.find_by_tag("GroupBy#3").state().loc[0,'__state__'].state()
type(s)

pandas.core.series.Series

In [32]:
pip_fit.find_by_tag("GroupBy#3").state().assign(
    mean = lambda df: df['__state__'].map(lambda x: x.state()[0])
)

,cut,__state__,mean
0,Ideal,"FitDeMean(tag='DeMean#3', cols=['price'], w_col=None, <21551 rows of fitting data>)",3457.541970
1,Premium,"FitDeMean(tag='DeMean#3', cols=['price'], w_col=None, <13791 rows of fitting data>)",4584.257704
2,Good,"FitDeMean(tag='DeMean#3', cols=['price'], w_col=None, <4906 rows of fitting data>)",3928.864452
3,Very Good,"FitDeMean(tag='DeMean#3', cols=['price'], w_col=None, <12082 rows of fitting data>)",3981.759891
4,Fair,"FitDeMean(tag='DeMean#3', cols=['price'], w_col=None, <1610 rows of fitting data>)",4358.757764


In [24]:
pip = (
    ff.Pipeline()
    #.group_by("cut", fitting_schedule=ff.fit_group_on_all_other_groups)
    .group_by("cut")
        .stateless_lambda(lambda df: df[["price"]].mean())
)
result = pip.fit_and_apply(df).set_index('cut').sort_index().reset_index()#.sort_values('cut')

In [20]:
target = df.groupby("cut")[['price']].mean().sort_index().reset_index()

In [26]:
result.equals(target)

True

In [25]:
result

,cut,price
0,Fair,4358.757764
1,Good,3928.864452
2,Ideal,3457.541970
3,Premium,4584.257704
4,Very Good,3981.759891


In [21]:
target

,cut,price
0,Fair,4358.757764
1,Good,3928.864452
2,Ideal,3457.541970
3,Premium,4584.257704
4,Very Good,3981.759891


In [40]:
pip = (
    ff.Pipeline()
    .group_by("cut", fitting_schedule=ff.fit_group_on_all_other_groups)
    #.group_by("cut")
        .de_mean('price')
    [['cut', 'price']]
)
result = pip.fit_and_apply(df)
result

,cut,price
0,Ideal,-3923.026830
1,Premium,-3383.026850
2,Good,-3606.193458
3,Premium,-3375.026850
4,Good,-3598.193458
...,...,...
53935,Ideal,-1492.026830
53936,Good,-1176.193458
53937,Very Good,-1161.667734
53938,Premium,-952.026850


In [51]:
cuts = pd.Series(df['cut'].unique(), name='cut').sort_values()
cut_means = pd.DataFrame(dict(
    cut=cuts,
    price=cuts.map(lambda v: df.loc[df['cut'] != v]['price'].mean())
))
cut_means

,cut,price
4,Fair,3919.694573
2,Good,3933.193458
0,Ideal,4249.026830
1,Premium,3709.026850
3,Very Good,3918.667734


In [53]:
target = (
    df
    .merge(cut_means, how='left', on='cut', suffixes=('', '_mean'))
    .assign(price=lambda df: df['price'] - df['price_mean'])
    [['cut', 'price']]
)
target

,cut,price
0,Ideal,-3923.026830
1,Premium,-3383.026850
2,Good,-3606.193458
3,Premium,-3375.026850
4,Good,-3598.193458
...,...,...
53935,Ideal,-1492.026830
53936,Good,-1176.193458
53937,Very Good,-1161.667734
53938,Premium,-952.026850


In [54]:
result.equals(target)

True